<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Problems with the Classical Theory

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Problems

Topics of interest include:

* extreme allocations
* loss of diversification
* portfolio turnover over time
* sensitivity to changes in return expectations

## Real Financial Data

**_Historical end-of-day financial time series data._**

See Artificial Intelligence in Finance (ch. 04)  and `http://hilpisch.com/aiif_eikon_eod_data.csv`.

## Imports and Data

In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
pd.set_option("display.precision", 5)
np.set_printoptions(suppress=True,
        formatter={'float': lambda x: f'{x:.4f}'})

In [ ]:
raw = pd.read_csv('http://hilpisch.com/aiif_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [ ]:
rets = np.log(raw / raw.shift(1)).dropna()

## Portfolio Statistics

In [ ]:
def port_return(rets, weights):
    return np.dot(rets.mean(), weights) * 252  # annualized

In [ ]:
def port_volatility(rets, weights):
    return np.dot(weights, np.dot(rets.cov() * 252 , weights)) ** 0.5  # annualized

In [ ]:
def port_sharpe(rets, weights):
    return port_return(rets, weights) / port_volatility(rets, weights)

## Extreme Allocations

In [ ]:
from scipy.optimize import minimize

In [ ]:
symbols = rets.columns[:3]
symbols

### No Short Sales

In [ ]:
bnds = len(symbols) * [(0, 1),]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
opt_weights = {}
for year in range(2010, 2020):
    rets_ = rets[symbols].loc[f'{year - 1}-01-01':f'{year}-12-31']
    ow = minimize(lambda weights: -port_sharpe(rets_, weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons)['x']
    opt_weights[year] = ow
ow_ = pd.DataFrame(opt_weights).T
ow_.columns = symbols

In [ ]:
opt_weights

In [ ]:
ow_.plot.bar(stacked=True, alpha=0.75);

### No Short Sales, Lower Bounds

In [ ]:
bnds = len(symbols) * [(0.1, 1),]
bnds

In [ ]:
opt_weights = {}
for year in range(2010, 2020):
    rets_ = rets[symbols].loc[f'{year - 1}-01-01':f'{year}-12-31']
    ow = minimize(lambda weights: -port_sharpe(rets_, weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons)['x']
    opt_weights[year] = ow
ow = pd.DataFrame(opt_weights).T
ow.columns = symbols

In [ ]:
opt_weights

In [ ]:
ow.plot.bar(stacked=True, alpha=0.75);

### No Short Sales, Upper Bounds

In [ ]:
bnds = len(symbols) * [(0, 0.8),]
bnds

In [ ]:
opt_weights = {}
for year in range(2010, 2020):
    rets_ = rets[symbols].loc[f'{year - 1}-01-01':f'{year}-12-31']
    ow = minimize(lambda weights: -port_sharpe(rets_, weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons)['x']
    opt_weights[year] = ow
ow = pd.DataFrame(opt_weights).T
ow.columns = symbols

In [ ]:
opt_weights

In [ ]:
ow.plot.bar(stacked=True, alpha=0.75);

### Short Sales, No Bounds

In [ ]:
opt_weights = {}
for year in range(2010, 2020):
    rets_ = rets[symbols].loc[f'{year - 1}-01-01':f'{year}-12-31']
    ow = minimize(lambda weights: -port_sharpe(rets_, weights),
                  len(symbols) * [1 / len(symbols)],
                  # bounds=bnds,
                  # constraints=cons
                 )['x']
    opt_weights[year] = ow
ow = pd.DataFrame(opt_weights).T
ow.columns = symbols

In [ ]:
opt_weights

In [ ]:
ow.plot.bar(stacked=True, alpha=0.75);

## Loss of Diversification

In [ ]:
weights = len(symbols) * [1 / len(symbols)]  # reference portfolio (equal weights)
weights

In [ ]:
ow_.plot.bar(stacked=True, alpha=0.75);

In [ ]:
# zero MSE means a "perfectly diversified" portfolio = exactly the reference portfolio
# the higher the MSE, the less diversified (the further away from the reference)
((ow_ - weights) ** 2).mean(axis=1)

In [ ]:
ow.plot.bar(stacked=True, alpha=0.75);

In [ ]:
((ow - weights) ** 2).mean(axis=1)

## Portfolio Turnover

In [ ]:
ow_.plot.bar(figsize=(10, 6));  # positions

In [ ]:
ow_.diff().plot.bar(figsize=(10, 6));  # changes in positions

In [ ]:
ow.plot.bar(figsize=(10, 6));  # positions

In [ ]:
ow.diff().plot.bar(figsize=(10, 6));  # changes in positions

## Sensitivity to Changes in Predictions

In [ ]:
def port_return(rets_mean, weights):
    return np.dot(rets_mean, weights) * 252  # annualized

In [ ]:
bnds = len(symbols) * [(0, 1),]
bnds

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
rets_mean = rets.mean()
rets_mean

In [ ]:
opt_weights = {}
for adj in np.linspace(-0.1, 0.1, 11):
    rets_mean_ = rets_mean.copy()
    rets_mean_.iloc[0] += adj / 252
    # rets_mean_.iloc[1] -= adj / 252
    ow = minimize(lambda weights: -port_return(rets_mean_[symbols], weights) /
                                  port_volatility(rets[symbols], weights),
                  len(symbols) * [1 / len(symbols)],
                  bounds=bnds,
                  constraints=cons
                 )['x']
    opt_weights[adj] = ow
ow = pd.DataFrame(opt_weights).T
ow.columns = symbols

In [ ]:
ow.round(5)

In [ ]:
ow.T.plot.bar(figsize=(10, 6), legend=False);  # positions

In [ ]:
ow.diff().T.plot.bar(figsize=(10, 6), legend=False);  # changes in positions

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>